In [131]:
import tensorflow as tf
import os
from PIL import Image

In [132]:
from google.colab import drive
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam

In [133]:
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [134]:
path = "/content/drive/MyDrive/Deep Learning/images"
file_list = os.listdir(path)
train_dataset = path + "/train"
validation_dataset = path + "/validation"


In [135]:
# def verify_images_in_directory(directory):
#     invalid_images = []
#     for root, _, files in os.walk(directory):
#         for file in files:
#             file_path = os.path.join(root, file)
#             try:
#                 img = Image.open(file_path)
#                 img.verify()  # Verifies that it is, in fact, an image
#             except (IOError, SyntaxError) as e:
#                 print(f"Invalid image file: {file_path} - {e}")
#                 invalid_images.append(file_path)
#     return invalid_images

In [136]:
# Verifikasi gambar dalam direktori train dan validation
# invalid_train_images = verify_images_in_directory(train_dataset)
# invalid_val_images = verify_images_in_directory(validation_dataset)

# print(f"Invalid images in train directory: {invalid_train_images}")
# print(f"Invalid images in validation directory: {invalid_val_images}")


In [137]:
train_subdirs = os.listdir(train_dataset)
val_subdirs = os.listdir(validation_dataset)

In [138]:
num_classes = len(train_subdirs)
assert num_classes == len(val_subdirs), "Number of classes in training and validation directories must match."

print(f"Number of classes: {num_classes}")

Number of classes: 7


In [139]:
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

Num GPUs Available:  0


In [140]:
# set img width
img_width, img_height = 256, 256
batch_size = 52

In [141]:
train_datagen = ImageDataGenerator(rescale=1./255)
validation_datagen = ImageDataGenerator(rescale=1./255)

In [142]:
train_generator = train_datagen.flow_from_directory(
    train_dataset,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical')

validation_generator = validation_datagen.flow_from_directory(
    validation_dataset,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical')

Found 34546 images belonging to 7 classes.
Found 8978 images belonging to 7 classes.


In [143]:
# Create CNN
model = Sequential()

model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(img_width, img_height, 3)))
model.add(MaxPooling2D((2, 2)))

In [144]:
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))

model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))

In [145]:
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(7, activation='softmax'))

In [146]:
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

Train Model

In [147]:
from tensorflow.keras.callbacks import TensorBoard
tensorboard_callback = TensorBoard(log_dir='./logs', histogram_freq=1)

In [ ]:
epochs = 20
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // batch_size,
    callbacks=[tensorboard_callback]
    )

Epoch 1/20
  8/664 [..............................] - ETA: 5:23:12 - loss: 2.7205 - accuracy: 0.1755

In [ ]:
import matplotlib.pyplot as plt

In [ ]:

# Plot training & validation accuracy values
plt.figure(figsize=(12, 4))

plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')

# Plot training & validation loss values
plt.subplot(1, 2, 2)
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')

plt.tight_layout()
plt.show()

Validation

In [ ]:
test_loss, test_acc = model.evaluate(validation_generator)
print('Test accuracy:', test_acc)

In [ ]:
model_save_path = "/content/drive/MyDrive/Deep Learning/emotion_detection_model.h5"
model.save(model_save_path)

testing

In [ ]:
test_images_dir = path + "/testing"
file_list = os.listdir(test_images_dir)
print(file_list )

In [ ]:
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
img_width = 48
img_height = 48
class_labels = ['bahagia', 'marah', 'sedih', 'terkejut', 'takut', 'jijik', 'netral']

In [ ]:
model_path = "/content/drive/MyDrive/Deep Learning/emotion_detection_model.h5"
model = load_model(model_path)

In [ ]:
def predict_image(image_path):
    img = load_img(image_path, target_size=(img_width, img_height))
    img_array = img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = img_array / 255.0  # Normalisasi gambar

    predictions = model.predict(img_array)
    predicted_class = np.argmax(predictions, axis=1)
    return class_labels[predicted_class[0]]

# Step 7: Menampilkan gambar dengan prediksi
def display_predictions(test_images_dir, num_images=5):
    test_image_files = [f for f in os.listdir(test_images_dir) if os.path.isfile(os.path.join(test_images_dir, f))]
    selected_images = np.random.choice(test_image_files, num_images, replace=False)

    plt.figure(figsize=(15, 15))

    for i, image_file in enumerate(selected_images):
        image_path = os.path.join(test_images_dir, image_file)
        predicted_label = predict_image(image_path)

        plt.subplot(1, num_images, i + 1)
        img = load_img(image_path, target_size=(img_width, img_height))
        plt.imshow(img)
        plt.title(f"Prediksi: {predicted_label}")
        plt.axis('off')

    plt.show()



In [ ]:
# Step 7: Melakukan prediksi untuk setiap gambar dalam folder testing
test_image_files = [f for f in os.listdir(test_images_dir) if os.path.isfile(os.path.join(test_images_dir, f))]
for image_file in test_image_files:
    image_path = os.path.join(test_images_dir, image_file)
    predicted_label = predict_image(image_path)
    print(f"Gambar: {image_file} -> Prediksi: {predicted_label}")

In [ ]:
display_predictions(test_images_dir, num_images=7)